# Predicting Stock Price

* Companies use Software to Predict Movements in Stocks

* This Program will Scrape a given Amount of Stocks from the Web, 

* Predict their Price in a Set Number of Days and Send an SMS to the User,

* Informing them of Stocks that might be Good to Check out and Invest in.

In [1]:
import numpy as np
from datetime import datetime
import smtplib
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import os

# For Prediction
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import svm, preprocessing

# For Stock Data
from iexfinance.stocks import get_historical_data

## Steps to Follow :

Using the **Selenium Package** we can **Scrape Yahoo Finance Stock Screeners** (Aggresive Small Cap Stocks).

Function to Retrieve Stocks.

In [2]:
def getstocks(n): # n : The Number of Stocks we Wish to Retrieve.
    
    # Creating Web Driver to Retrieve Web Page.  
    driver = webdriver.Chrome(ChromeDriverManager().install()) 

    # URL of Yahoo Finance
    url = "https://finance.yahoo.com/screener/predefined/aggressive_small_caps?offset=0&count=202"

    # Retrieve the Web Page by it's URL.
    driver.get(url) 

    # Creating a Stock List
    stock_list = []
    n += 1

    # Iterating through Each Symbol Tickers
    for i in range(1, n):
        ticker = driver.find_element_by_xpath('//*[@id="scr-res-table"]/div[1]/table/tbody/tr['+str(i)+']/td[1]/a')
        stock_list.append(ticker.text)

    driver.quit() # Exit Web Browser

    # Using Stock List to Predict Future Price of the Stock
    number = 0

    for i in stock_list:
        print(f'Number : {str(number)}')
        try:
            predictData(i,5) # 5 Days
        except:
            print(f'Stock : {i} was not Predicted')
        number += 1    

## Predict the Stocks

* Using 3 Years of Data for Prediction

* **iexfinance** Function to get **Historical Data** of the Stocks.

In [3]:
def predictData(stock, days):

    print(stock)
    
    start = datetime(2017,1,1)
    
    end = datetime.now()
    
    data = get_historical_data(stock, start=start, end=end, output_format='pandas')
    
    if os.path.exists('./Exports'): # If Path Exist then Save the CSV File
    
        csv_file = ('Exports/' + stock + '_Export.csv')
        
    else:
        
        os.mkdir('Exports') # Create a Directory named Exports and Save the CSV File
        
        csv_file = ('Exports/' + stock + '_Export.csv')
        
    data.to_csv(csv_file)
        
    data['Prediction'] = data['close'].shift(-1)
    
    data.dropna(inplace = True)
    
    forecast = int(days)
    
    # Predicting Stock Price in the Future.

    x = np.array(data.drop(['Prediction'],1))
    y = np.array(data['Prediction'])
    x = preprocessing.scale(x)
    x_prediction = x[-forecast] # X_Test Data
    
    # Splitting Data into Train Set and Test Set
    x_train, y_train, y_test = train_test_split(x, y, test_size=0.5)
    
    # Creating Linear Regression Model
    model = LinearRegression()
    
    # Fitting Model on Train and Test Set
    model.fit(x_train, y_train)
    
    # Prediction of Model on X_Test
    prediction = model.predict(x_prediction)
    
    last_row = data.tail(1)
    
    print(last_row['close'])

    # Sending SMS if Predicted Price of Stock is atleast 1 Greater than Previous Closing Price.
    if (float(prediction[4]) > (float(last_row['close'])) + 1):
        output = ("\n\nStock:" + str(stock) + "\nPrior Close:\n" + str(last_row['close']) + "\n\nPrediction in 1 Day: " + str(prediction[0]) + "\nPrediction in 5 Days: " + str(prediction[4]))

        sendMessage(output)


## Send Message

**GMAIL :**

* Go to **Security Settings** of your **Email Account** and **Enable** the **"Allow Less Secure Apps"** Option.

In [4]:
def  send_message(text):
    username = "abc"
    password = "abc@123"
    vtext = "abc@vtext.com"
    message = text

    msg = """From : %s
    To : %s
    %s"""%(username, vtext, message)

    server = smtplib.SMTP('smtp.gmail.com', 587) # Default Mail Submission Port
    server.starttls() # Protocol Command to Inform Mail Server for Secure Connection.
    server.login(username, password)
    server.sendmail(username, vtext, msg)
    server.quit()

    print(f'Mail Sent')

In [5]:
if __name__ == '__main__':
    getstocks(200)

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
 
[WDM] - There is no [win32] chromedriver for browser 84.0.4147 in cache
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/84.0.4147.30/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\30865\.wdm\drivers\chromedriver\win32\84.0.4147.30]
Number : 0
ASNA
Stock : ASNA was not Predicted
Number : 1
ACB
Stock : ACB was not Predicted
Number : 2
RIG
Stock : RIG was not Predicted
Number : 3
HMHC
Stock : HMHC was not Predicted
Number : 4
KOS
Stock : KOS was not Predicted
Number : 5
HEXO
Stock : HEXO was not Predicted
Number : 6
APHA
Stock : APHA was not Predicted
Number : 7
YRCW
Stock : YRCW was not Predicted
Number : 8
VAL
Stock : VAL was not Predicted
Number : 9
CBL
Stock : CBL was not Predicted
Number : 10
EXK
Stock : EXK was not Predicted
Number : 11
OIBR-C
Stock : OIBR-C was not Predicte